# **Installing the needed libs**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install youtube-mp3-downloader
!pip install youtube-transcript-api
!pip install youtube_dl
!pip install pydub

# **Get Videos IDs given text file**

In [ ]:
def get_videos_ID(filename):
  return open(filename).read().splitlines()

# **Get subtitles given video ID**

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
def get_subtitles(video_id):
  try:
    subtitles = YouTubeTranscriptApi.get_transcript(video_id, languages=['en'])
    return subtitles
  except:
    return None

# **Download video in mp3 given video ID**

In [ ]:
import youtube_dl
from __future__ import unicode_literals


In [ ]:
def download_mp3(video_id):
  ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
          'preferredquality': '192',
      }],
  }
  url = f"http://www.youtube.com/watch?v={video_id}"
  with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      try:
        ydl.download([url])
      except:
        return None

  return 1


# **Read, Write and hear mp3**

In [ ]:
import pydub 
import numpy as np
from IPython.display import Audio

In [ ]:
def read(f, normalized=False):
    """MP3 to numpy array"""
    a = pydub.AudioSegment.from_mp3(f)
    y = np.array(a.get_array_of_samples())
    if a.channels == 2:
        y = y.reshape((-1, 2))
        y = y.sum(axis=1) / 2
    if normalized:
        return a.frame_rate, np.float32(y) / 2**15
    else:
        return a.frame_rate, y

def write(f, sr, x, normalized=False):
    """numpy array to MP3"""
    channels = 2 if (x.ndim == 2 and x.shape[1] == 2) else 1
    if normalized:  # normalized array - each item should be a float in [-1, 1)
        y = np.int16(x * 2 ** 15)
    else:
        y = np.int16(x)
    song = pydub.AudioSegment(y.tobytes(), frame_rate=sr, sample_width=2, channels=channels)
    song.export(f, format="mp3", bitrate="320k")
    return


In [ ]:
def hear_audio(track, sr):
  Audio(track,rate=sr,autoplay=True)
  return

In [ ]:
track.shape

# **Cutting sound according to subtitles**

In [ ]:
subtitles

In [ ]:
# sr, track, subtitles

slices = []
slices_text = []

for subtitle in subtitles:
  text = subtitle['text']
  text = text.replace('\n', ' ')
  start = int(subtitle['start'] * sr)
  end = int(start + subtitle['duration'] * sr)

  slice_track = track[start:min(end, track.shape[0])]

  slices.append(slice_track)
  slices_text.append(text)

  #print(text)
  #print(start)
  #print(duration)

  #break

In [ ]:
sample = 17

In [ ]:
max(map(len, slices_text))

In [ ]:
slices_text[sample]

In [ ]:
Audio(slices[sample],rate=sr,autoplay=True)


# **Block merging for data collecting**